<h1>第八周作业<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#第八周作业运行结果" data-toc-modified-id="第八周作业运行结果-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>第八周作业运行结果</a></span><ul class="toc-item"><li><span><a href="#densenet实现" data-toc-modified-id="densenet实现-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>densenet实现</a></span></li><li><span><a href="#运行" data-toc-modified-id="运行-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>运行</a></span><ul class="toc-item"><li><span><a href="#在quiz数据集上" data-toc-modified-id="在quiz数据集上-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>在quiz数据集上</a></span></li><li><span><a href="#在cifar10数据集上" data-toc-modified-id="在cifar10数据集上-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>在cifar10数据集上</a></span></li></ul></li></ul></li></ul></div>

# 第八周作业运行结果

## densenet实现

In [1]:
def densenet(images, num_classes=1001, is_training=False,
             dropout_keep_prob=0.8,
             scope='densenet'):
    """Creates a variant of the densenet model.

      images: A batch of `Tensors` of size [batch_size, height, width, channels].
      num_classes: the number of classes in the dataset.
      is_training: specifies whether or not we're currently training the model.
        This variable will determine the behaviour of the dropout layer.
      dropout_keep_prob: the percentage of activation values that are retained.
      prediction_fn: a function to get predictions out of logits.
      scope: Optional variable_scope.

    Returns:
      logits: the pre-softmax activations, a tensor of size
        [batch_size, `num_classes`]
      end_points: a dictionary from components of the network to the corresponding
        activation.
    """
    growth = 24
    compression_rate = 0.5
    
    def reduce_dim(input_feature):  #压缩, 增加模型紧凑性
        return int(int(input_feature.shape[-1]) * compression_rate)

    end_points = {}

    with tf.variable_scope(scope, 'DenseNet', [images, num_classes]):
        with slim.arg_scope(bn_drp_scope(is_training=is_training,
                                         keep_prob=dropout_keep_prob)) as ssc:
            pass
            ##########################
            # 244 x 244 x 3
            end_point = 'Conv_0'
            net = slim.conv2d(images, 2*growth, [7,7], stride=2, padding='SAME', scope=end_point)
            end_point = 'Pool_0'
            # 112 x 112 x 48
            net = slim.max_pool2d(net, [3,3], stride=2, padding='SAME', scope=end_point)
            end_points[end_point] = net
            # 56 x 56 x 48
            for i in range(4):
                end_point = 'dense_{}'.format(i+1)
                net = block(net, 6, reduce_dim(net), scope=end_point) #拼接
                net = bn_act_conv_drp(net, (i+1)*8, [1,1], scope=end_point)  #非线性变换
                end_points[end_point] = net
                
            # 56 x 56 x 16
            end_point = 'logits'
            net_shape = net.get_shape().as_list()
            # global_avg_pool2d
            net = slim.avg_pool2d(net, net_shape[1:3], scope=end_point)
            # => [batch_size, 1, 1, num_classes]
            net = slim.conv2d(net, num_classes, [1, 1], activation_fn=None,
                             normalizer_fn=None, scope=end_point)
            # => [batch_size, `num_classes`]
            logits = tf.squeeze(net, [1, 2], name=end_point)
            end_points[end_point] = logits
            
            ##########################

    return logits, end_points

## 运行

### 在quiz数据集上

tinymin地址:https://www.tinymind.com/executions/4p6vaurk (运行4000step后killed)

正确率的计算: https://www.tinymind.com/executions/jb70ace6

```
运行结果:
2018-04-20 09:42:58.665508: I tensorflow/core/kernels/logging_ops.cc:79]eval/Accuracy[0.015625]     
2018-04-20 09:42:58.665549: I tensorflow/core/kernels/logging_ops.cc:79]eval/Recall_5[0.046875]     
INFO:tensorflow:Finished evaluation at 2018-04-20-09:42:58
```

### 在cifar10数据集上

tinymin地址: https://www.tinymind.com/executions/wksh6vxy (运行10个epoch)

正确率的计算: https://www.tinymind.com/executions/c8vtg4bb

```
运行结果:
2018-04-21 08:58:19.495556: I tensorflow/core/kernels/logging_ops.cc:79] eval/Accuracy[0.2193]
2018-04-21 08:58:19.495556: I tensorflow/core/kernels/logging_ops.cc:79] eval/Recall_5[0.7116]
INFO:tensorflow:Finished evaluation at 2018-04-21-08:58:19
```